In [1]:
import nltk
#nltk.download()

In [2]:
import requests
import pandas as pd
import re, string, unicodedata
from pprint import pprint
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk import FreqDist
from textblob import TextBlob

#import contractions
#import inflect

In [3]:
violence_words = set(["asesinato", "asesino", "asesina", "matar", "mata", "mato", "apuñalar","apuñalo","pega","pego","apalizo","golpea","golpeo","agredio","agrede"])

In [4]:
print(violence_words)

{'pega', 'pego', 'asesina', 'asesino', 'apalizo', 'apuñalar', 'golpea', 'golpeo', 'agrede', 'mato', 'asesinato', 'agredio', 'apuñalo', 'mata', 'matar'}


In [5]:
stopWords = set(stopwords.words('spanish'))

In [6]:
print(stopWords)

{'e', 'seremos', 'estuviera', 'mí', 'habiendo', 'su', 'esos', 'como', 'tenían', 'habrías', 'habida', 'éramos', 'teniendo', 'estados', 'tienen', 'está', 'fuésemos', 'estuvieron', 'fuesen', 'esté', 'fueran', 'ellas', 'serían', 'hubieseis', 'poco', 'sus', 'tuyos', 'donde', 'habías', 'estemos', 'tuvo', 'pero', 'tendrían', 'tenida', 'hayamos', 'esta', 'habréis', 'tengas', 'hubieran', 'estuvo', 'tuvieran', 'muchos', 'a', 'nuestro', 'estadas', 'un', 'ese', 'os', 'habrás', 'suyos', 'nuestros', 'nosotros', 'seríamos', 'estará', 'estáis', 'fuéramos', 'qué', 'hubiste', 'has', 'sentidas', 'habidos', 'tuvisteis', 'algunas', 'tuvieseis', 'hubieses', 'haya', 'habré', 'habían', 'algunos', 'había', 'hubiéramos', 'ha', 'tenido', 'otras', 'estaréis', 'vuestras', 'le', 'seáis', 'hayan', 'hubiera', 'estabas', 'tendrá', 'otra', 'ella', 'están', 'esa', 'teníamos', 'con', 'sí', 'y', 'estarás', 'fuimos', 'hemos', 'los', 'les', 'tendrán', 'estuvieseis', 'sentido', 'estarías', 'estuve', 'no', 'fuese', 'tuvieron'

In [7]:
MADRID_URL = "https://www.madridiario.es/sucesos"

In [8]:
def get_soup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser") #parser para que no se queje
    return soup

soup = get_soup(MADRID_URL)

In [9]:
#Get headers and its publication date
def get_headers(soup):
    headers = []
    date = []
    
    container = soup.find_all('div',{'class','n1 fueraNoticia'})
    for item in container:
        headers.append(item.find('h2',{'class','titulo'}).text)
        date.append(item.find('meta',itemprop='datePublished').get('content')[:10])

    if len(headers) == len(date):
        return pd.DataFrame({"headers": headers, "datePublication": date})
    else:
        return -1 #exit with error because there is news with date

        
dfnews_raw = get_headers(soup)
dfnews_raw.head(11)

,headers,datePublication
0,Apuñalamiento mortal en un parque de Valdezarza,2019-06-28
1,Herida grave al volcar su coche en Los Santos ...,2019-06-28
2,Un joven en silla de ruedas cae a las vías en ...,2019-06-28
3,Desalojado un edificio por un incendio que dej...,2019-06-28
4,Anticorrupción pide 7 años de cárcel para Calv...,2019-06-28
5,Desarticulada una red de venta de cocaína en M...,2019-06-28
6,Agresión a un empleado cuando cambiaba la cerr...,2019-06-27
7,Muere un mecánico al caerle un coche encima en...,2019-06-27
8,Vuelca con un coche robado en su huída de la G...,2019-06-27
9,Un incendio destruye una nave industrial en Hu...,2019-06-27


In [ ]:
"""
I will try to do all in just one function called clean

for i in range(0,len(df['tweet'])):
			# get rid of anythin that isnt a letter

			exclusion_list = ['[^a-zA-Z]','rt', 'http', 'co', 'RT']
			exclusions = '|'.join(exclusion_list)
			text = re.sub(exclusions, ' ' , df['tweet'][i])
			text = text.lower()
			words = text.split()
			words = [word for word in words if not word in stopword_list]
			 # only use stem of word
			#words = [ps.stem(word) for word in words]
			df['clean_tweets'][i] = ' '.join(words)
"""

In [10]:
# Get rid of lower-cases
def tolower (s):
    return s.lower()


dfnews_raw.headers = dfnews_raw.headers.apply(tolower)
dfnews_raw.head(6)

,headers,datePublication
0,apuñalamiento mortal en un parque de valdezarza,2019-06-28
1,herida grave al volcar su coche en los santos ...,2019-06-28
2,un joven en silla de ruedas cae a las vías en ...,2019-06-28
3,desalojado un edificio por un incendio que dej...,2019-06-28
4,anticorrupción pide 7 años de cárcel para calv...,2019-06-28
5,desarticulada una red de venta de cocaína en m...,2019-06-28


In [11]:
# Get rid of accents
def clean_accent(s):
    return s.replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u')

dfnews_raw.headers = dfnews_raw.headers.apply(clean_accent)
dfnews_raw.head(6)

,headers,datePublication
0,apuñalamiento mortal en un parque de valdezarza,2019-06-28
1,herida grave al volcar su coche en los santos ...,2019-06-28
2,un joven en silla de ruedas cae a las vias en ...,2019-06-28
3,desalojado un edificio por un incendio que dej...,2019-06-28
4,anticorrupcion pide 7 años de carcel para calv...,2019-06-28
5,desarticulada una red de venta de cocaina en m...,2019-06-28


In [12]:
#trying to pass severals parameters without functools and operator
#dfnews_raw.headers = dfnews_raw.headers.apply(clean_any_character(work_list,','))
#dfnews_raw.headers = dfnews_raw.headers.apply(clean_charac, character=',')
#dfnews_raw.headers = dfnews_raw.headers.apply(clean_charac, arg=(dfnews_raw,',')


In [13]:
#Get rid of commas
def clean_commas(s):
    return s.replace(',','')

dfnews_raw.headers = dfnews_raw.headers.apply(clean_commas)
dfnews_raw.head(11)

,headers,datePublication
0,apuñalamiento mortal en un parque de valdezarza,2019-06-28
1,herida grave al volcar su coche en los santos ...,2019-06-28
2,un joven en silla de ruedas cae a las vias en ...,2019-06-28
3,desalojado un edificio por un incendio que dej...,2019-06-28
4,anticorrupcion pide 7 años de carcel para calv...,2019-06-28
5,desarticulada una red de venta de cocaina en m...,2019-06-28
6,agresion a un empleado cuando cambiaba la cerr...,2019-06-27
7,muere un mecanico al caerle un coche encima en...,2019-06-27
8,vuelca con un coche robado en su huida de la g...,2019-06-27
9,un incendio destruye una nave industrial en hu...,2019-06-27


In [19]:
#Get rid of quotes
def clean_quotes(s):
    return s.replace('\"','').replace('\'','')

dfnews_raw.headers = dfnews_raw.headers.apply(clean_quotes)

dfnews_clean_string = dfnews_raw.copy() #copy for sentiment study
dfnews_clean_token = dfnews_raw.copy() #copy for word frequency study
dfnews_clean_string.head(18)

,headers,datePublication
0,apuñalamiento mortal en un parque de valdezarza,2019-06-28
1,herida grave al volcar su coche en los santos ...,2019-06-28
2,un joven en silla de ruedas cae a las vias en ...,2019-06-28
3,desalojado un edificio por un incendio que dej...,2019-06-28
4,anticorrupcion pide 7 años de carcel para calv...,2019-06-28
5,desarticulada una red de venta de cocaina en m...,2019-06-28
6,agresion a un empleado cuando cambiaba la cerr...,2019-06-27
7,muere un mecanico al caerle un coche encima en...,2019-06-27
8,vuelca con un coche robado en su huida de la g...,2019-06-27
9,un incendio destruye una nave industrial en hu...,2019-06-27


In [17]:
# Get each word / Tokenizar
def token(s):
    return nltk.word_tokenize(s)

dfnews_clean_token.headers = dfnews_clean_string.headers.apply(token)
dfnews_clean_token.head(6)

,headers,datePublication
0,"[apuñalamiento, mortal, en, un, parque, de, va...",2019-06-28
1,"[herida, grave, al, volcar, su, coche, en, los...",2019-06-28
2,"[un, joven, en, silla, de, ruedas, cae, a, las...",2019-06-28
3,"[desalojado, un, edificio, por, un, incendio, ...",2019-06-28
4,"[anticorrupcion, pide, 7, años, de, carcel, pa...",2019-06-28
5,"[desarticulada, una, red, de, venta, de, cocai...",2019-06-28


In [ ]:
def clean_stopwords(lst):
    
    [row.remove(item) for row in list_tokens for item in row if item in stp_words]
    return list_tokens
    
cleaned_data = clean_stopwords(token_news, stopWords)
pprint(cleaned_data[:6])

In [ ]:


#new = new + " enfado"
new = [' '.join(item) for item in cleaned_data]


for item in new:
    if TextBlob(item).detect_language() == 'es':
        print("1")
        analysis = TextBlob(str(item))
        item = analysis.translate(to='spanish')
        print(item)
    analysis = TextBlob(str(item))
    print(analysis.sentiment,"\n")




"""
new = ' '.join(cleaned_data[1])
print(new)
print(TextBlob(new).detect_language())
analysis = TextBlob(str(new))
print(analysis.sentiment,"\n")


new = ' '.join(cleaned_data[2])
print(new)
print(TextBlob(new).detect_language())
analysis = TextBlob(str(new))
print(analysis.sentiment,"\n")

"""
"""
new2 = analysis.translate(to='spanish')
analysis = TextBlob(str(new2))
print(analysis.sentiment)
print(analysis.tags)
#print(analysis.translate(to='spanish'))
print(dir(analysis))
"""


In [ ]:
def new_item(list_tokens):
    for index, row in enumerate(list_tokens):
        list_tokens.insert(index+1, [])
        list_tokens.insert(index+1, [])
    return list_tokens
 
print(new_item(cleaned_data))

In [ ]:
def violence_at_row(list_tokens,set_list):
    for row in list_tokens:
        for word in row:
            if word in violence_words:
                

In [ ]:
for word in cleaned_data:
    if 